# Comparison results

In [2]:
# Imports
import os
from pathlib import Path
from datetime import datetime
import pandas as pd

# Constants
ALIGNERS = ['vgaligner', 'graphaligner', 'graphchainer', 'vgmap', 'giraffe']

In [3]:
DATASETS = [name for name in os.listdir(".") if os.path.isdir(name) and not name.startswith(".")]
DATASETS.remove("scripts")
DATASETS

['E3133-pggb']

In [4]:
results = dict()
for dataset in DATASETS:
    results_folder = os.path.join(".", dataset, "results")
    
    results_by_path = dict()
    for name in os.listdir(results_folder):
        name_without_ext = name[:-4]
        path,aligner = name_without_ext.split('_')
        
        if path not in results_by_path.keys():
            results_by_path[path] = []
        results_by_path[path].append((aligner,os.path.join(results_folder, name)))
    
    results[dataset] = results_by_path

# Q1: Which aligners worked and which didn't?

In [5]:
for dataset in DATASETS:
    print('Results for {}:'.format(dataset))
    results_by_path = results[dataset]
    for path in results_by_path.keys():
        '''
        print('Path {} aligned by {}'.format(
            path, 
            list(map(lambda result : result[0], results_by_path[path]))
        ))
        '''
        print('Path {} aligned by {} aligners'.format(
            path, 
            len(results_by_path[path])
        ))
        
        if len(results_by_path[path]) < len(ALIGNERS):
           print('Path {} NOT aligned by {}'.format(
            list(set(ALIGNERS)-set(results_by_path[path]))
        )) 

Results for E3133-pggb:
Path gi|568815592:30489405-30494204 aligned by 6 aligners
Path gi|568815529:1969140-1973939 aligned by 6 aligners
Path gi|568815551:1745237-1750036 aligned by 6 aligners
Path gi|568815569:1790174-1794973 aligned by 6 aligners
Path gi|568815564:1833461-1838260 aligned by 6 aligners
Path gi|568815567:1744501-1749300 aligned by 6 aligners
Path gi|568815561:1799640-1804439 aligned by 6 aligners
Path gi|157734152:30257311-30262110 aligned by 6 aligners
Path gi|528476637:30459287-30464086 aligned by 6 aligners


In [6]:
aligners_results = []
for dataset in DATASETS:
    curr_record = dict()
    
    results_by_path = results[dataset]
    for path in results_by_path.keys():
        curr_record['dataset'] = dataset
        #curr_record['n aligner mapped'] = len(results_by_path[path])
        curr_record['mapped by'] = set(map(lambda result : result[0], results_by_path[path]))
        curr_record['not mapped by'] = set(ALIGNERS)-curr_record['mapped by']
        curr_record['% aligner that worked'] = len(results_by_path[path])/len(ALIGNERS) * 100
        aligners_results.append(curr_record)
        # TODO: remove .gaf from vg map (in snakefile)
        
aligners_results_df = pd.DataFrame(aligners_results)
aligners_results_df

,dataset,mapped by,not mapped by,% aligner that worked
0,E3133-pggb,"{vgmap, giraffe, graphaligner, graphchainer, v...",{},120.0
1,E3133-pggb,"{vgmap, giraffe, graphaligner, graphchainer, v...",{},120.0
2,E3133-pggb,"{vgmap, giraffe, graphaligner, graphchainer, v...",{},120.0
3,E3133-pggb,"{vgmap, giraffe, graphaligner, graphchainer, v...",{},120.0
4,E3133-pggb,"{vgmap, giraffe, graphaligner, graphchainer, v...",{},120.0
5,E3133-pggb,"{vgmap, giraffe, graphaligner, graphchainer, v...",{},120.0
6,E3133-pggb,"{vgmap, giraffe, graphaligner, graphchainer, v...",{},120.0
7,E3133-pggb,"{vgmap, giraffe, graphaligner, graphchainer, v...",{},120.0
8,E3133-pggb,"{vgmap, giraffe, graphaligner, graphchainer, v...",{},120.0


# Q2: How did the aligners perform? 

In [7]:
time_by_datasets = dict()
memory_by_datasets = dict()

for dataset in DATASETS:
    dataset_time_by_path = dict()
    dataset_memory_by_path = dict()
    
    path_time_by_aligner = dict()
    path_memory_by_aligner = dict()
    
    logs_folder = os.path.join(".", dataset, "logs")
    
    for name in os.listdir(logs_folder): 
        
        # TODO: fix in snakefile
        if "vgaligner_map" in name:
            name = name.replace("vgaligner_map", "vgalignermap")
        
        name_without_ext = name[:-4] if name.endswith('.log') else name[:-5]
        
        if name_without_ext == 'vgindex':
            #print(name_without_ext)
            continue
        else:
            path,aligner = name_without_ext.split('_')
            
            # TODO: fix in snakemake
            if "vgalignermap" in name:
                name = name.replace("vgalignermap", "vgaligner_map")
            
            log_full_path = os.path.join(logs_folder, name)
            with open(log_full_path, "r") as fp:
                for line in fp.readlines():
                    
                    '''
                    if line.lstrip().startswith("System time"):
                        time = float(line.split(':')[1])
                        #print(time)
                    '''
                    
                    # TODO in the future: this could either be hh:mm:ss OR mm:ss.ms
                    if line.lstrip().startswith("Elapsed (wall clock) time"):
                        elapsed_time_str = line.lstrip()[45:]
                        path_time_by_aligner[aligner] = datetime.strptime(elapsed_time_str.strip(), '%M:%S.%f').time()
                        #print(elapsed_time_str.rstrip())
                        #print(datetime.strptime(elapsed_time.strip(), '%M:%S.%f').time())
                    
                    if line.lstrip().startswith("Maximum resident set size"):
                        space = int(line.split(':')[1])
                        path_time_by_aligner[aligner] = space
                        datetime.strptime(elapsed_time_str.strip(), '%M:%S.%f').time()
                        #print(space, "kb")
            
            dataset_memory_by_path[path] = path_memory_by_aligner
            dataset_time_by_path[path] = path_time_by_aligner
    
    time_by_datasets[dataset] = dataset_time_by_path
    memory_by_datasets[dataset] = dataset_memory_by_path
    
print(time_by_datasets)
print(memory_by_datasets)

{'E3133-pggb': {'gi|568815569:1790174-1794973': {'graphchainer': 28272, 'graphaligner': 21456, 'vgalignermap': 2135196, 'vgmap': 22040, 'index': 8544}, 'gi|157734152:30257311-30262110': {'graphchainer': 28272, 'graphaligner': 21456, 'vgalignermap': 2135196, 'vgmap': 22040, 'index': 8544}, 'gi|568815561:1799640-1804439': {'graphchainer': 28272, 'graphaligner': 21456, 'vgalignermap': 2135196, 'vgmap': 22040, 'index': 8544}, 'gi|568815592:30489405-30494204': {'graphchainer': 28272, 'graphaligner': 21456, 'vgalignermap': 2135196, 'vgmap': 22040, 'index': 8544}, 'gi|568815567:1744501-1749300': {'graphchainer': 28272, 'graphaligner': 21456, 'vgalignermap': 2135196, 'vgmap': 22040, 'index': 8544}, 'gi|568815564:1833461-1838260': {'graphchainer': 28272, 'graphaligner': 21456, 'vgalignermap': 2135196, 'vgmap': 22040, 'index': 8544}, 'gi|568815551:1745237-1750036': {'graphchainer': 28272, 'graphaligner': 21456, 'vgalignermap': 2135196, 'vgmap': 22040, 'index': 8544}, 'gi|568815529:1969140-197393

In [8]:
aligners_performance = []

for dataset in DATASETS:
    
    logs_folder = os.path.join(".", dataset, "logs")
    for name in os.listdir(logs_folder):
        
        if not name.endswith(".time"):
            continue
        
        # TODO: fix in snakefile
        if "vgaligner_map" in name:
            name = name.replace("vgaligner_map", "vgalignermap")
        
        name_without_ext = name[:-5]
        
        if name_without_ext == 'vgindex':
            continue
        else:
            path,aligner = name_without_ext.split('_')
            
            # TODO: fix in snakemake
            if "vgalignermap" in name:
                name = name.replace("vgalignermap", "vgaligner_map")
            
            log_full_path = os.path.join(logs_folder, name)
            with open(log_full_path, "r") as fp:
                curr_record = dict()
                curr_record['dataset'] = dataset
                curr_record['aligner'] = aligner
                curr_record['path'] = name.split('_')[0]
                for line in fp.readlines():
                    
                    # TODO in the future: this could either be hh:mm:ss OR mm:ss.ms
                    if line.lstrip().startswith("Elapsed (wall clock) time"):
                        elapsed_time_str = line.lstrip()[45:]
                        curr_record['time'] = datetime.strptime(elapsed_time_str.strip(), '%M:%S.%f').time()
                    
                    if line.lstrip().startswith("Maximum resident set size"):
                        space = int(line.split(':')[1])
                        curr_record['space']= space
                
                aligners_performance.append(curr_record)
    
aligners_performance_df = pd.DataFrame(aligners_performance)
group_by = aligners_performance_df.groupby(['dataset','path','aligner'])
group_by.first()

time  \
dataset    path                           aligner                         
E3133-pggb gi|157734152:30257311-30262110 graphaligner  00:00:00.050000   
                                          graphchainer  00:00:00.060000   
                                          vgalignermap  00:00:00.850000   
                                          vgmap         00:00:06.600000   
           gi|528476637:30459287-30464086 graphaligner  00:00:00.050000   
                                          graphchainer  00:00:00.060000   
                                          vgalignermap  00:00:00.930000   
                                          vgmap         00:00:06.350000   
           gi|568815529:1969140-1973939   graphaligner  00:00:00.050000   
                                          graphchainer  00:00:00.050000   
                                          vgalignermap  00:00:00.890000   
                                          vgmap         00:00:06.590000   
           gi|568815551:1745237-1750036   graphaligner  00:00:00.050000   
                                          graphchainer  00:00:00.050000   
                                          vgalignermap  00:00:00.780000   
                                          vgmap         00:00:06.400000   
           gi|568815561:1799640-1804439   graphaligner  00:00:00.050000   
                                          graphchainer  00:00:00.060000   
                                          vgalignermap  00:00:00.930000   
                                          vgmap         00:00:06.250000   
           gi|568815564:1833461-1838260   graphaligner  00:00:00.050000   
                                          graphchainer  00:00:00.050000   
                                          vgalignermap  00:00:00.870000   
                                          vgmap         00:00:06.400000   
           gi|568815567:1744501-1749300   graphaligner  00:00:00.050000   
                                          graphchainer  00:00:00.060000   
                                          vgalignermap  00:00:00.900000   
                                          vgmap         00:00:06.500000   
           gi|568815569:1790174-1794973   graphaligner  00:00:00.050000   
                                          graphchainer  00:00:00.060000   
                                          vgalignermap  00:00:00.870000   
                                          vgmap         00:00:06.480000   
           gi|568815592:30489405-30494204 graphaligner  00:00:00.050000   
                                          graphchainer  00:00:00.050000   
                                          vgalignermap  00:00:00.780000   
                                          vgmap         00:00:06.550000   
           vgaligner                      index         00:00:00.110000   

                                                          space  
dataset    path                           aligner                
E3133-pggb gi|157734152:30257311-30262110 graphaligner    21336  
                                          graphchainer    28272  
                                          vgalignermap  2135112  
                                          vgmap           22040  
           gi|528476637:30459287-30464086 graphaligner    21152  
                                          graphchainer    27516  
                                          vgalignermap  2132988  
                                          vgmap           22888  
           gi|568815529:1969140-1973939   graphaligner    21316  
                                          graphchainer    27652  
                                          vgalignermap  2135196  
                                          vgmap           22236  
           gi|568815551:1745237-1750036   graphaligner    21876  
                                          graphchainer    27912  
                                          vgalignermap  2136024  
                   

## Q3: Parse graphs stats

In [9]:
graphs = list()
for dataset in DATASETS:
    curr_graph_stats = dict()
    curr_graph_stats["name"] = dataset
    
    stats_file = os.path.join(".", dataset, "stats", "stats_{}.txt".format(dataset))
    with open(stats_file, "r") as fp:
        for line in fp.readlines():
            if line.startswith("nodes"):
                curr_graph_stats["nodes"] = int(line.split("\t")[1])
            elif line.startswith("edges"):
                curr_graph_stats["edges"] = int(line.split("\t")[1])
            elif line.startswith("self-loops"):
                curr_graph_stats["self-loops"] = int(line.split("\t")[1])
            else:
                curr_graph_stats["cyclic"] = True if line == "cyclic" else False
    
    graphs.append(curr_graph_stats)
    
graphs_df = pd.DataFrame(graphs)
graphs_df

,name,nodes,edges,self-loops,cyclic
0,E3133-pggb,13,16,0,False


# Q4: Parse comparison results

In [26]:
jaccard_results = list()
for dataset in DATASETS:
    curr_graph_stats = dict()
    curr_graph_stats["name"] = dataset
    
    comparisons_folder = os.path.join(".", dataset, "comparisons")
    for name in os.listdir(comparisons_folder):
        name_without_ext = name[:-4]
        path,aligner = name_without_ext.split('_')
        
        comparison_file = os.path.join(".", dataset, "comparisons", name)
        with open(comparison_file, "r") as fp:
            for line in fp.readlines():
                if line.lstrip().startswith("Reads mapped correctly"):
                    _,value = line.lstrip().split(":")
                    absolute, _ = value.lstrip().split(" ")
                    n_mapped, total_reads = absolute.split("/")
                    curr_result = {
                        'name':dataset,
                        'aligner':aligner,
                        'path': path,
                        'n_mapped_correctly':int(n_mapped), 
                        'total_reads': int(total_reads),
                        '% correct': int(n_mapped)/int(total_reads)
                    }
                    break
        
        jaccard_results.append(curr_result)

jaccard_results_df = pd.DataFrame(jaccard_results)
group_by = jaccard_results_df.groupby(['name','path','aligner'])
group_by.first()

n_mapped_correctly  \
name       path                           aligner                            
E3133-pggb gi|157734152:30257311-30262110 giraffe                        6   
                                          graphaligner                  51   
                                          graphchainer                  36   
                                          vgaligner                     36   
                                          vgmap                         18   
           gi|528476637:30459287-30464086 giraffe                        6   
                                          graphaligner                  51   
                                          graphchainer                  36   
                                          vgaligner                     36   
                                          vgmap                         18   
           gi|568815529:1969140-1973939   giraffe                        6   
                                          graphaligner                  51   
                                          graphchainer                  36   
                                          vgaligner                     36   
                                          vgmap                         18   
           gi|568815551:1745237-1750036   giraffe                        6   
                                          graphaligner                  51   
                                          graphchainer                  36   
                                          vgaligner                     36   
                                          vgmap                         18   
           gi|568815561:1799640-1804439   giraffe                        6   
                                          graphaligner                  51   
                                          graphchainer                  36   
                                          vgaligner                     36   
                                          vgmap                         18   
           gi|568815564:1833461-1838260   giraffe                        6   
                                          graphaligner                  51   
                                          graphchainer                  36   
                                          vgaligner                     36   
                                          vgmap                         18   
           gi|568815567:1744501-1749300   giraffe                        6   
                                          graphaligner                  51   
                                          graphchainer                  36   
                                          vgaligner                     36   
                                          vgmap                         18   
           gi|568815569:1790174-1794973   giraffe                        6   
                                          graphaligner                  51   
                                          graphchainer                  36   
                                          vgaligner                     36   
                                          vgmap                         18   
           gi|568815592:30489405-30494204 giraffe                        6   
                                          graphaligner                  51   
                                          graphchainer                  36   
                                          vgaligner                     36   
                                          vgmap                         18   

                                                        total_reads  % correct  
name       path                           aligner                               
E3133-pggb gi|157734152:30257311-30262110 giraffe                16      0.375  
                                          graphaligner           51      1.000  
                                          graphchainer           